In [1]:
import os
import torch
from torch import nn
import json
import faiss
import numpy as np
import pickle
import math
import re
os.environ['CUDA_VISIBLE_DEVICES']="0"
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer, AutoModel,AutoConfig

In [2]:
model_name="/root/autodl-tmp/model/text2vec"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
print(tokenizer)
print(model)

BertTokenizer(name_or_path='/root/autodl-tmp/model/text2vec', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, 

In [4]:
def normal(vector):
    ss=sum([s**2 for s in vector])
    return [round(s/ss,5) for s in vector]
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    # print("token_embeddings")
    # print(token_embeddings)
    # print(len(token_embeddings),token_embeddings.size())
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    # print("input_mask_expanded")
    # print(input_mask_expanded)
    result= torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)#求平均
    # print(torch.sum(token_embeddings * input_mask_expanded, 1))
    # print(torch.clamp(input_mask_expanded.sum(1), min=1e-9))
    # print(result.shape)
    result=result[0].tolist()
    # print(len(result))
    result=normal(result)
    return result
def get_vector(sentence):
    encoded_input = tokenizer([sentence], padding=True, truncation=True, return_tensors='pt')
    # print("encoded_input")
    # print(encoded_input)
    # print(tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True))
    model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    return sentence_embeddings

# 建立知识向量库

In [5]:
id_knowadge={}
with open("/root/autodl-tmp/data/wenlv/wenlv.jsonl",encoding="utf-8") as f:
    lines=[json.loads(s.strip()) for s in f.readlines()]
id_vector=[]
print(len(lines))
for i,data in enumerate(lines):
    #if(i%100==0): print(i)
    query=data["context"].replace("Instruction: ","").replace("\nAnswer: ","")
    target=data["target"]
    #使用bert模型把问题进行向量化
    vector=get_vector(query)
    # #每条知识，对应一个id
    # #id和向量
    id_vector.append(vector)
    # #id和对应的知识
    id_knowadge[i]={"query":query,"target":target}
id_vector=np.array(id_vector)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


2956


In [6]:
# print(id_vector[0])
# print(id_vector[0].dtype)
id_vector = id_vector.astype(np.float32)
# print(id_vector_32[0])
# print(id_vector_32[0].dtype)
with open("id_knowadge","w") as f:
    json.dump(id_knowadge,f,ensure_ascii=False)
#id和向量用faiss库来存储
#faiss是向量数据库，支持向量的高性能检索
index = faiss.IndexFlatL2(768) 
# print(index.is_trained)
index.add(id_vector)    
# print(index.ntotal)
with open("id_vector","wb") as f:
    pickle.dump(index,f)

# LangChain推理比较

In [7]:
glm_tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/model/chatglm", trust_remote_code=True)
glm_model = AutoModel.from_pretrained("/root/autodl-tmp/model/chatglm", trust_remote_code=True).float()
glm_config = AutoConfig.from_pretrained( "/root/autodl-tmp/model/chatglm", trust_remote_code=True, device_map='auto')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
def get_prompt(sentences,query):
    return "请根据以下事实回答问题{}。问题是：{}".format("。".join(sentences),query)
with open("id_vector","rb") as f:
    index=pickle.load(f)
with open("id_knowadge") as f:
    id_know=json.load(f)

In [17]:
input="梦里老家实景演出票价"
#把问题变成向量
vector = get_vector(input)
vector = np.array([vector])
vector = vector.astype(np.float32)
#从faiss库里面找到最近的5个id  D距离，I id
D, I = index.search(vector, 5)
D=D[0]
I=I[0]
sentences=[]
for d,i in zip(D,I):
    #距离过滤
    if d>0.02:
        continue
    #print (id_know[str(i)]['query'])
    sentences.append(id_know[str(i)]['target'])
print(sentences)

['《梦里老家》实景演出门票价格：238元', '上饶市婺源县紫阳镇梦里老家文旅小镇', '《梦里老家》实景演出位于上饶市婺源县紫阳镇梦里老家文旅小镇。', '3月4日-12月31日20:00-21:10(停止售票19:45,最晚入园19:55)', '《梦里老家》实景演出']


In [20]:
prompt="梦里老家实景演出票价和演出时间"
print (prompt)
response, history = glm_model.chat(glm_tokenizer, prompt, history=[])
print(response)
print("________________________________")
prompt=get_prompt(sentences,prompt)
print (prompt)
response, history = glm_model.chat(glm_tokenizer, prompt, history=[])
print(response)
print("Hello World")

梦里老家实景演出票价和演出时间
我无法提供具体的梦里老家实景演出的票价和演出时间，因为这些信息可能会随着演出地点、演出时间和其他因素的变化而有所不同。建议您查询相关演出网站或联系售票机构以获取最新的票价和演出时间信息。
________________________________
请根据以下事实回答问题《梦里老家》实景演出门票价格：238元。上饶市婺源县紫阳镇梦里老家文旅小镇。《梦里老家》实景演出位于上饶市婺源县紫阳镇梦里老家文旅小镇。。3月4日-12月31日20:00-21:10(停止售票19:45,最晚入园19:55)。《梦里老家》实景演出。问题是：梦里老家实景演出票价和演出时间
梦里老家实景演出的票价为238元，演出时间为3月4日至12月31日，演出时间为20:00至21:10，停止售票时间为19:45，最晚入园时间为19:55。
